In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from scipy.stats import norm
import os

if os.path.exists('Results'):
    pass
else:
    os.makedirs('Results')


In [ ]:
# paramter set up 
s0=100
r=0.05
vol=0.10
T=2
noofsteps=100
noofpaths=10000

def monte_carlo(s0,r,T,vol,noofsteps,noofpaths):
    dt=T/noofsteps
    z=np.random.standard_normal((noofpaths,noofsteps))
    rets = (r - 0.5*vol**2)*dt + vol*z*np.sqrt(dt)
    cum_rets = np.cumsum(rets,axis=1)
    paths = np.exp(cum_rets)*s0
    paths = np.hstack([s0*np.ones((noofpaths,1)),paths])
    time = np.arange(0,noofsteps+1)*dt
    return paths,time
    
def American_option_pricer(s0,strike,vol,r,T,noofsteps,noofpaths,type):
    paths,time =monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
    type = type.lower()
    dt= T/noofsteps
    excercise_matrix=np.zeros((noofpaths,noofsteps+1))
    cashflows = np.zeros((noofpaths,noofsteps+1))
    discount = np.exp(-r*dt)
    Excercise_payoff_immediate = np.maximum(paths-strike,0) if type=='call' else np.maximum(strike-paths,0)
    cashflows[:,-1] = Excercise_payoff_immediate[:,-1]
    excercise_matrix[:,-1] = Excercise_payoff_immediate[:,-1] > 0
    for t in range(noofsteps-1,0,-1):
                itm = Excercise_payoff_immediate[:,t]>0
                x=paths[itm,t]
                y= cashflows[itm,t+1]*discount
                if len(x)==0:
                    cashflows[:, t] = cashflows[:, t+1] * discount
                    continue
                
                coeffs = np.polyfit(x,y,2)
                continuation= np.polyval(coeffs,x)
                excercise = Excercise_payoff_immediate[itm,t] > continuation
                itm_indices = np.where(itm)[0]
                excercise_matrix[itm_indices[excercise], t] = 1
                rows = itm_indices[excercise]
                cashflows[rows,t] = Excercise_payoff_immediate[rows,t]
                cashflows[rows,t+1:] = 0
                cashflows[itm_indices[~excercise],t] = cashflows[itm_indices[~excercise],t+1]*discount
                not_itm = ~itm
                cashflows[not_itm,t] = cashflows[not_itm,t+1]*discount
                excercise_matrix[itm_indices[excercise],t+1:] = 0
    price =  np.mean(cashflows[:,1]*discount)
    
    return price,excercise_matrix
    
paths,time=monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
price,exercised=American_option_pricer(s0,100,vol,r,T,noofsteps,noofpaths,'put')

def BSM(s,k,r,t,vol):
    d1 = (np.log(s/k) + (r+t*0.5*vol**2))/(vol*np.sqrt(t))
    d2 = d1-vol*np.sqrt(t)
    price = s*norm.cdf(d1) - k*np.exp(-r*t)*norm.cdf(d2)
    return price
BSM(s0,100,r,T,vol)
American_option_pricer(s0,80,vol,r,T,noofsteps,noofpaths,'call')[0] , BSM(s0,80,r,T,vol)

In [ ]:
T=1
k=np.linspace(90,140,20)
p_euro = [BSM(s0,i,r,T,vol) - 0.5 for i in k] 
p_american = [American_option_pricer(s0,i,vol,r,T,noofsteps,noofpaths,'call')[0] for i in k]
p_american > p_euro


In [ ]:
# Importnant plots 

# plot_1 number of paths and simulation
N=50
idx = np.random.choice(paths.shape[0],N,replace=False)
plt.style.use('dark_background')
plt.figure(figsize=(10,6))

for i in idx:
    plt.plot(time,paths[i,:],color='cornflowerblue',alpha=0.5)
plt.axhline(100,0,color='orange')
plt.xlabel('Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.ylabel('Number of paths',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.title('Monte carlo simulated paths ',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.grid(color='Red',alpha=0.3)
plt.tight_layout()
plt.legend()
plt.savefig('Results/Path_GBM',bbox_inches='tight',dpi=300)
    


#plot 2 EXCERICSE BOUNDARY VISAULIZATION
# Ensure time is 1D with length paths.shape[1]
times=np.argmax(exercised,axis=1)
row = np.arange(exercised.shape[0])

has_exercised =exercised.sum(axis=1)==1

row_ind= row[has_exercised]
col_ind = times[has_exercised]
s=paths[exercised.astype(bool)]
s=np.where(s>100,s,np.nan)

t=time[col_ind]

plt.figure(figsize=(9, 5))
plt.scatter(t, s, s=8, alpha=0.5, color='#FF5733', label='Exercise')
plt.title('American Option: Exercise Events Across Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.xlabel('Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.ylabel('Underlying Price at Exercise',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('Results/Optimal Exercise Domain',bbox_inches='tight',dpi=300)



# plotting the european and americian option prices for the given strike and durations
plt.figure(figsize=(10,6))
plt.plot(k,p_euro,label='European Call Price',color='cyan')
plt.plot(k,p_american,label='American Call Price',color='yellow')
plt.xlabel('strikes',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.ylabel('Price',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.title('Comparative Analyis for European/American Options',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.legend()
plt.tight_layout()
plt.grid(color='red',alpha=0.4)
plt.savefig('Results/European - American call Price comparison',bbox_inches='tight',dpi=300)

